In [17]:
# PNC

import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/PNC_Good/AngleBasisLong20.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

basis = pickle.load(open(basis_file, 'rb'))

thetas = []
jitter = []
age = []
sex = []
race = []
fc = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

for subtask in basis:
    m = re.search('([^-]+)-(.+)', subtask)
    sub = m.group(1)
    task = m.group(2)
    if sub not in demo['Race'] or demo['Race'][sub] not in ['AA', 'EA']:
        continue
    a = demo['age_at_cnb'][sub]
    s = demo['Sex'][sub] == 'M'
    r = demo['Race'][sub] == 'AA'
    age.append(a)
    sex.append(s)
    race.append(r)
    thetas.append(basis[subtask]['thetas'])
    jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-{task}_fc.npy')
    fc.append(p)
    
thetas = np.stack(thetas)
jitter = np.stack(jitter)
age = np.array(age).astype('int')
sex = np.array(sex).astype('int')
race = np.array(race).astype('int')
fc = np.stack(fc)

print([a.shape for a in [thetas, jitter, sex, race, age, fc]])
print(np.mean(1-sex))
print(np.mean(1-race))
print(np.mean(age))

[(3849, 20, 264), (3849, 20, 264), (3849,), (3849,), (3849,), (3849, 34716)]
0.5263704858404781
0.5188360613146272
14.398285268901013


In [18]:
def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def tops(thetas, jitter):
    t0 = np.expand_dims(thetas, 2)
    t1 = np.expand_dims(thetas, 3)
    j0 = np.expand_dims(jitter, 2)
    j1 = np.expand_dims(jitter, 3)
    ps = np.cos(t0-t1)*(j0*j1)
    a,b = np.triu_indices(264, 1)
    ps = ps[:,:,a,b]
    return ps

paps = []
pres = []

for i in range(0,fc.shape[0],500):
    ps = tops(thetas[i:i+500], jitter[i:i+500])
    aps = np.mean(ps, axis=1)
    res = fc[i:i+500] - aps
    paps.append(aps)
    pres.append(res)
    print(f'Done {i}')
    
aps = np.concatenate(paps)
res = np.concatenate(pres)

print(ps.shape)
print(aps.shape)
print(res.shape)

Done 0
Done 500
Done 1000
Done 1500
Done 2000
Done 2500
Done 3000
Done 3500
(349, 20, 34716)
(3849, 34716)
(3849, 34716)


In [19]:
aps20 = aps

print('Done')

Done


In [21]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

def tonp(x):
    return x.detach().cpu().numpy()

def totorch(x):
    return torch.from_numpy(x).float().cuda()

def totorchidcs(x):
    return torch.from_numpy(x).long().cuda() #F.one_hot(torch.from_numpy(x)).float().cuda()

def rmse(yt, yhat):
    return torch.mean((yt-yhat)**2)**0.5

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(34716,100).float().cuda()
        self.l2 = nn.Linear(100,2).float().cuda()
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        return x
    
def fit_mlp(xtr, ytr, verbose=False):
    xtr = totorch(xtr)
    ytr = totorchidcs(ytr)
    
    ce = nn.CrossEntropyLoss()
    mlp = MLP()
    optim = torch.optim.Adam(mlp.parameters(), lr=5e-4, weight_decay=5e-4)

    nepochs = 1000
    pperiod = 100

    for epoch in range(nepochs):
        optim.zero_grad()
        yhat = mlp(xtr)
        loss = ce(yhat, ytr)
        loss.backward()
        optim.step()
        if verbose:
            if epoch % pperiod == 0 or epoch == nepochs-1:
                print(f'{epoch} {float(loss)}')

    if verbose:
        print('Complete')
    
    return mlp
    
def predict_help(model, xt):
    xt = totorch(xt)
    with torch.no_grad():
        yhat = model(xt)
        return tonp(yhat)
    
print('Done')

Done


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import roc_auc_score

def cat(x, **kwargs):
    return np.concatenate(x, **kwargs)

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def predict(xtr, xt, ytr, yt, lst):
    reg = fit_mlp(xtr, ytr)
    p = predict_help(reg, xt)
    acc = roc_auc_score(yt, p[:,1])
    print(acc)
    lst.append(acc)
    return p
    
def get_res(fctr, fct, abtr, abt):
    return fctr-abtr, fct-abt

def combine(yt, p0, p1, lst):
    p = (p0+p1)/2
    acc = roc_auc_score(yt, p[:,1])
    print(acc)
    lst.append(acc)

rfc = []

rab1 = []
rab3 = []
rab5 = []
rab10 = []
rab15 = []
rab20 = []

rres1 = []
rres3 = []
rres5 = []
rres10 = []
rres15 = []
rres20 = []

rens1 = []
rens3 = []
rens5 = []
rens10 = []
rens15 = []
rens20 = []

rbest = []

for i in range(10):

    x0tr, x0t, x1tr, x1t, x2tr, x2t, x3tr, x3t, x4tr, x4t, x5tr, x5t, x6tr, x6t, ytr, yt = train_test_split(
        fc, aps1, aps3, aps5, aps10, aps15, aps20, sex, stratify=sex, train_size=0.8)
    
    x1atr, x1at = get_res(x0tr, x0t, x1tr, x1t)
    x2atr, x2at = get_res(x0tr, x0t, x2tr, x2t)
    x3atr, x3at = get_res(x0tr, x0t, x3tr, x3t)
    x4atr, x4at = get_res(x0tr, x0t, x4tr, x4t)
    x5atr, x5at = get_res(x0tr, x0t, x5tr, x5t)
    x6atr, x6at = get_res(x0tr, x0t, x6tr, x6t)

    predict(x0tr, x0t, ytr, yt, rfc)
    
    p1 = predict(x1tr, x1t, ytr, yt, rab1)
    p2 = predict(x2tr, x2t, ytr, yt, rab3)
    p3 = predict(x3tr, x3t, ytr, yt, rab5)
    p4 = predict(x4tr, x4t, ytr, yt, rab10)
    p5 = predict(x5tr, x5t, ytr, yt, rab15)
    p6 = predict(x6tr, x6t, ytr, yt, rab20)
    
    p1a = predict(x1atr, x1at, ytr, yt, rres1)
    p2a = predict(x2atr, x2at, ytr, yt, rres3)
    p3a = predict(x3atr, x3at, ytr, yt, rres5)
    p4a = predict(x4atr, x4at, ytr, yt, rres10)
    p5a = predict(x5atr, x5at, ytr, yt, rres15)
    p6a = predict(x6atr, x6at, ytr, yt, rres20)
    
    combine(yt, p1, p1a, rens1)
    combine(yt, p2, p2a, rens3)
    combine(yt, p3, p3a, rens5)
    combine(yt, p4, p4a, rens10)
    combine(yt, p5, p5a, rens15)
    combine(yt, p6, p6a, rens20)

    combine(yt, p6, p1a, rbest)
    
    print('---')
    
print(np.mean(rfc), np.std(rfc))
print(np.mean(rbest), np.std(rbest))

print(np.mean(rab1), np.std(rab1))
print(np.mean(rab3), np.std(rab3))
print(np.mean(rab5), np.std(rab5))
print(np.mean(rab10), np.std(rab10))
print(np.mean(rab15), np.std(rab15))
print(np.mean(rab20), np.std(rab20))

print(np.mean(rres1), np.std(rres1))
print(np.mean(rres3), np.std(rres3))
print(np.mean(rres5), np.std(rres5))
print(np.mean(rres10), np.std(rres10))
print(np.mean(rres15), np.std(rres15))
print(np.mean(rres20), np.std(rres20))

print(np.mean(rens1), np.std(rens1))
print(np.mean(rens3), np.std(rens3))
print(np.mean(rens5), np.std(rens5))
print(np.mean(rens10), np.std(rens10))
print(np.mean(rens15), np.std(rens15))
print(np.mean(rens20), np.std(rens20))

0.9277794689666836
0.8017791307289024
0.8673769660071029
0.7998308811094199
0.9158599695585998
0.9300186030779638
0.9283341789277861
0.9525114155251142
0.9509216979536615
0.9468831388466094
0.9434737020125148
0.9345577541011332
0.9046034162015897
0.9504346355487908
0.9505766954168781
0.9101978691019785
0.9645729748012853
0.965499746321664
0.9563064434297309
0.9641332656857772
---
0.9434601725012683
0.7924978860138677
0.8540842212075089
0.8366852697446305
0.9284965330627432
0.9323930323017081
0.9359512937595129
0.9603450025367833
0.958532048029765
0.9558532048029764
0.955900558092339
0.9330559783527821
0.9294503636056147
0.9545881955014375
0.9583832234060544
0.9498393370539491
0.9660679857940132
0.9578149839337055
0.9627059022492812
0.9684018264840182
---
0.9348689328598005
0.8014950109927279
0.8754473194655843
0.8974598342634871
0.9150008455944529
0.9284085912396414
0.9031692880094706
0.9488584474885845
0.9542093691865382
0.9487705056654828
0.9414375105699306
0.9188905800777946
0.91824